In [56]:
import logging
import os

import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import swyft
from scipy.stats import multivariate_normal

import NSLFI.NestedSampler
from NSLFI.NRE_Settings import NRE_Settings
from NSLFI.Swyft_NRE_Wrapper import NRE


In [57]:
np.random.seed(234)
logging.basicConfig(filename="myLFI.log", level=logging.INFO,
                    filemode="w")
logger = logging.getLogger()
logger.info('Started')
root = "swyft_data"
try:
    os.makedirs(root)
except OSError:
    logger.info("root folder already exists!")

nreSettings = NRE_Settings(base_path=root)
nreSettings.n_training_samples = 10_00
nreSettings.n_weighted_samples = 10_000
nreSettings.trainmode = True
#NS rounds, 0 is default NS run
rounds = 2
# define forward model dimensions
nParam = 2
nData = 3
# true parameters of simulator
cov = 0.01 * np.eye(nData)
mu = 0.5 * np.ones(nParam)
F = np.eye(nData, nParam)
x0 = multivariate_normal.rvs(F @ mu, cov)

#uniform prior for theta_i
lower = 0
upper = 1
theta_prior = stats.uniform(loc=lower, scale=upper)
#wrap prior for NS sampling procedure
prior = {f"theta_{i}": theta_prior for i in range(nParam)}

In [58]:
class Simulator(swyft.Simulator):
    def __init__(self):
        super().__init__()
        self.transform_samples = swyft.to_numpy32

    def build(self, graph):
        means = graph.node('means', lambda: theta_prior.rvs(size=nParam))
        x = graph.node('x',
                       lambda means: multivariate_normal.rvs(F @ means, cov),
                       means)


sim = Simulator()
samples = sim.sample(N=nreSettings.n_training_samples)

100%|██████████| 4000/4000 [00:01<00:00, 3425.71it/s]


In [59]:
 # initialize swyft network
class Network(swyft.SwyftModule):
    def __init__(self):
        super().__init__()
        marginals = (tuple(x for x in range(nParam)),)
        # self.logratios1 = swyft.LogRatioEstimator_1dim(num_features = 1, num_params = 3, varnames = 'z')
        self.logratios2 = swyft.LogRatioEstimator_Ndim(num_features=nData, marginals=marginals,
                                                       varnames='means',
                                                       hidden_features=32)

    def forward(self, A, B):
        # logratios1 = self.logratios1(A['x'], B['z'])
        logratios2 = self.logratios2(A['x'], B['means'])
        return logratios2


trainer = swyft.SwyftTrainer(accelerator=nreSettings.device, devices=1, max_epochs=20, precision=64,
                             enable_progress_bar=False, default_root_dir=nreSettings.base_path)
dm = swyft.SwyftDataModule(samples, fractions=[0.8, 0.02, 0.1], num_workers=3, batch_size=256)
network = Network()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
# train MRE
if nreSettings.trainmode is True:
    trainer.fit(network, dm)

    # load NRE from file
else:
    checkpoint_path = os.path.join(nreSettings.base_path,
                                   "lightning_logs/version_7/checkpoints/epoch=19-step=84.ckpt")
    network = network.load_from_checkpoint(checkpoint_path)

Missing logger folder: swyft_data_larger_prior_0to2_4000train/lightning_logs

  | Name       | Type                   | Params
------------------------------------------------------
0 | logratios2 | LogRatioEstimator_Ndim | 4.7 K 
------------------------------------------------------
4.7 K     Trainable params
0         Non-trainable params
4.7 K     Total params
0.038     Total estimated model params size (MB)
/Users/hikarukilian/IdeaProjects/NS_LFI/venv/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1558: PossibleUserWarning: The number of training batches (14) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


In [ ]:
 # get posterior samples
means = theta_prior.rvs(size=(nreSettings.n_weighted_samples, nParam))
B = swyft.Samples(means=means)
C = swyft.Sample(means=x0)
x0 = swyft.Sample(x=x0)
predictions = trainer.infer(network, x0, B)
# plot initial NRE

plt.figure()
labeler = {f"means[{i}]": fr"$\mu_{i}$" for i in range(nParam)}
swyft.corner(predictions, tuple(f"means[{i}]" for i in range(nParam)), labeler=labeler, bins=200, smooth=3);
plt.suptitle("NRE parameter estimation")
plt.savefig(fname=f"{root}/firstNRE.pdf")

In [ ]:
logProb_0 = trainer.infer(network, x0, C)
print(f"log probability of theta_0 using NRE is: {float(logProb_0.logratios):.3f}")

In [ ]:
# wrap NRE object
trained_NRE = NRE(network=network, trainer=trainer, prior=prior, nreSettings=nreSettings, obs=x0,
                  livepoints=means)

# wrap NRE for Polychord
poly_NRE = NRE_Poly(nre=trained_NRE.mre_2d, x0=x0)
polychordSet = PolyChordSettings(nParams=poly_NRE.nParams, nDerived=poly_NRE.nDerived)
polychordSet.nlive = n_training_samples
try:
    comm_analyse = MPI.COMM_WORLD
    rank_analyse = comm_analyse.Get_rank()
except Exception as e:
    logger.error(
        "Oops! {} occurred. when Get_rank()".format(e.__class__))
    rank_analyse = 0


def dumper(live, dead, logweights, logZ, logZerr):
    """Dumper Function for PolyChord for runtime progress access."""
    logger.info("Last dead point: {}".format(dead[-1]))


output = pypolychord.run_polychord(poly_NRE.loglike,
                                   poly_NRE.nParams,
                                   poly_NRE.nDerived,
                                   polychordSet,
                                   poly_NRE.prior, dumper)
comm_analyse.Barrier()


In [ ]:
output = NSLFI.NestedSampler.nested_sampling(logLikelihood=trained_NRE.logLikelihood,
                                             livepoints=trained_NRE.livepoints, prior=prior, nsim=100,
                                             stop_criterion=1e-3,
                                             samplertype="Metropolis", rounds=rounds, root=root)

In [ ]:
firstRoundPoints = means
secondRoundPoints = np.load(file=f"{root}/posterior_samples_rounds_0.npy")
secondRoundPoints_birthLog = np.load(file=f"{root}/logL_birth_rounds_0.npy")[0]
secondRoundPoints_LogLike = np.load(file=f"{root}/logL_rounds_0.npy")
thirdRoundPoints = np.load(file=f"{root}/posterior_samples_rounds_1.npy")
thirdRoundPoints_birthLog = np.load(file=f"{root}/logL_birth_rounds_1.npy")[0]
thirdRoundPoints_LogLike = np.load(file=f"{root}/logL_rounds_1.npy")

In [ ]:
np.median(secondRoundPoints_LogLike)

In [ ]:
thirdRoundPoints_LogLike.min()

In [ ]:
plt.figure()
plt.scatter(firstRoundPoints[:, 0], firstRoundPoints[:, 1], c="b", label="Round 1", s=3)
plt.scatter(secondRoundPoints[:, 0], secondRoundPoints[:, 1], c="r", label="Round 2", s=3)
plt.scatter(thirdRoundPoints[:, 0], thirdRoundPoints[:, 1], c="y", label="Round 3", s=3)
plt.legend()
plt.xlabel(r"$\theta_1$")
plt.ylabel(r"$\theta_2$")
plt.vlines(x=0.5, ymin=lower, ymax=upper, colors="cyan")
plt.hlines(y=0.5, xmin=lower, xmax=upper, colors="cyan")
plt.title(r"Log-weights contours using NS rounds, $\mathcal{L} > "
          r"\mathcal{L}_{\mathrm{median}}$")
plt.savefig(f"{root}/NS_rounds.pdf")